# Twitter Matching 

In [4]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import math

# File locations 
tweets_file_location = '~/Desktop/CS6242 Project/Processed_Data.csv'
runnerdata_file_location = '~/Desktop/CS6242 Project/FinalData.csv'

## Import Data

In [5]:
# Twitter Data
tweets = pd.read_csv(tweets_file_location,header = 0)
tweets.head()

,Unnamed: 0,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink,tidy_tweet,year,month
0,6146,1/31/2020 23:26,billstogsdill,BKSportsTalk,0,0,0,"Good luck to my AP having, RunDisney brother f...",NaN,NaN,NaN,1.220000e+18,https://twitter.com/billstogsdill/status/12233...,good luck ap rundisney brother another mother ...,2020,1
1,6147,1/31/2020 23:21,RunEatRepeat,NaN,0,0,1,Eat What You Want Day–Tinkerbell Run–Invisalig...,NaN,NaN,#TinkHalf #RunDisney #Tink10K #Tink5Kpic,1.220000e+18,https://twitter.com/RunEatRepeat/status/122338...,eat want day tinkerbell run invisalign update ...,2020,1
2,6148,1/31/2020 23:07,mattnonsense,NaN,1,0,0,Didn’t quite get the color right in my totally...,NaN,NaN,NaN,1.220000e+18,https://twitter.com/mattnonsense/status/122338...,quite get color right totally real rundisney beer,2020,1
3,6149,1/31/2020 22:46,brianflock,swart_jennifer,0,0,3,So much fun.pic.twitter.com/0HEUEOwSX3,NaN,NaN,NaN,1.220000e+18,https://twitter.com/brianflock/status/12233770...,much fun,2020,1
4,6150,1/31/2020 22:38,Jenlefforge,NaN,0,0,1,BTW it's possible the #RunDisney gods knew I w...,NaN,NaN,#RunDisney,1.220000e+18,https://twitter.com/Jenlefforge/status/1223375...,btw possible gods knew angered wanted make amends,2020,1


In [6]:
# Runner Data
runnerdata = pd.read_csv(runnerdata_file_location,header = 0)
runnerdata = runnerdata.drop(columns = ['Unnamed: 0'],axis = 1)
runnerdata.head()

,Race Index,Div Place,Name,Bib,Age,Age Group,Place,Gender Place,5 Mile Split,10 Mile Split,...,type,city,name,country,countryCode,region,regionCode,latitude,longitude,elevationMeters
0,0,1,FREDISON COSTA,1,38,35-39,1,1,1900-01-00 01:44:00,1900-01-02 04:20:00,...,0,0,0,0,0,0,0,0.000000,0.000000,0.0
1,1,2,MATEUS TRINDADE,13,31,30-34,2,2,1900-01-00 02:02:00,1900-01-02 06:47:00,...,CITY,Sapiranga,Sapiranga,Brazil,BR,Rio Grande do Sul,RS,-29.638060,-51.006940,63.0
2,2,3,JEFF MARTINEZ,20006,24,18-24,3,3,1900-01-00 06:23:00,01:00:18,...,CITY,Binghamton,Binghamton,United States of America,US,New York,NY,42.102222,-75.911667,866.0
3,3,1,JORDAN TROPF,36,24,18-24,4,4,1900-01-00 05:41:00,1900-01-02 11:10:00,...,CITY,Baltimore,Baltimore,Ireland,IE,Munster,M,51.483333,-9.366667,26.0
4,4,1,RYOSUKE SAITO,20020,30,30-34,5,5,1900-01-00 06:32:00,01:01:41,...,0,0,0,0,0,0,0,0.000000,0.000000,0.0


## Get Unique Runner Names and Twitter Usernames

In [7]:
# Get Unique Twitter Usernames
usernames = tweets['username'].unique()
usernames = pd.DataFrame(usernames)
usernames = usernames.rename(columns = {0:'username'})
usernames = usernames['username']#.array
usernames

0        billstogsdill
1         RunEatRepeat
2         mattnonsense
3           brianflock
4          Jenlefforge
             ...      
14799        mimiullom
14800    DavidChocarro
14801      lopez_juan2
14802     deborahsalmi
14803    Karenblankety
Name: username, Length: 14804, dtype: object

In [8]:
# Get unique runner names and set join key 
runnerdata_name = runnerdata['Name'].unique()
runnerdata_name = pd.DataFrame(runnerdata_name)
runnerdata_name = runnerdata_name.rename(columns = {0:'full name'})
runnerdata_name = runnerdata_name['full name']#.array
runnerdata_name

0         FREDISON COSTA
1        MATEUS TRINDADE
2          JEFF MARTINEZ
3           JORDAN TROPF
4          RYOSUKE SAITO
              ...       
63575    AMANDA BROCKMAN
63576       DALLAS DUKES
63577      HELENA WISTON
63578     DAWN MATTINSON
63579     PATRICIA SATOW
Name: full name, Length: 63580, dtype: object

## Split Data

In [9]:
# Get length
length = runnerdata_name.shape[0]
length_third_index = math.floor(length/4)
length_third_2_index = math.floor(length/4)*2
length_third_3_index = math.floor(length/4)*3

# Kevin's Part
runnerdata_name_1 = runnerdata_name.iloc[0:length_third_index]
runnerdata_name_1

# Hunter's Part
runnerdata_name_2 = runnerdata_name.iloc[length_third_index:length_third_2_index]
runnerdata_name_2

# James's Part
runnerdata_name_3 = runnerdata_name.iloc[length_third_2_index:length_third_3_index]
runnerdata_name_3

# Daniel's Part
runnerdata_name_4 = runnerdata_name.iloc[length_third_3_index:length]
runnerdata_name_4

# Part Input 
runnerdata_name_part = runnerdata_name_4
runnerdata_name_part


47685    DERRICK CONTRERAS
47686            ERIC YANG
47687        XIAOHANG PENG
47688      THALES COUCEIRO
47689      ROBERT MORRISON
               ...        
63575      AMANDA BROCKMAN
63576         DALLAS DUKES
63577        HELENA WISTON
63578       DAWN MATTINSON
63579       PATRICIA SATOW
Name: full name, Length: 15895, dtype: object

## Match twitter username based on name

In [10]:
def findtwitterhandle(x):
    #print(x)
    match = process.extractOne(x,usernames.array, scorer=fuzz.partial_token_sort_ratio)
    match = match if match[1] >= 80 else np.nan
    return match

In [11]:
runnerdata_with_usernames = pd.DataFrame(runnerdata_name_part)
runnerdata_with_usernames['match_and_score'] = runnerdata_name_part.apply(lambda x: findtwitterhandle(x))
runnerdata_with_usernames

,full name,match_and_score
47685,DERRICK CONTRERAS,NaN
47686,ERIC YANG,NaN
47687,XIAOHANG PENG,NaN
47688,THALES COUCEIRO,NaN
47689,ROBERT MORRISON,"(rzison, 83)"
...,...,...
63575,AMANDA BROCKMAN,"(amasna, 83)"
63576,DALLAS DUKES,NaN
63577,HELENA WISTON,NaN
63578,DAWN MATTINSON,NaN


In [12]:
runnerdata_with_usernames['score'] = runnerdata_with_usernames['match_and_score'].str[1]
runnerdata_with_usernames['twitter'] = runnerdata_with_usernames['match_and_score'].str[0]
runnerdata_with_usernames = runnerdata_with_usernames.sort_values(by=['score'], ascending = False)
runnerdata_with_usernames[runnerdata_with_usernames['score'] >= 80]

,full name,match_and_score,score,twitter
58868,ALLISON ASHURST,"(Allison, 100)",100.0,Allison
53420,JENNIFER CATANZARO,"(atanza, 100)",100.0,atanza
61419,ALLISON HUGHES,"(Allison, 100)",100.0,Allison
63377,ALLISON HALL,"(Allison, 100)",100.0,Allison
58772,ALLISON HUFFMAN,"(Allison, 100)",100.0,Allison
...,...,...,...,...
55212,FLAVIO RAMOS,"(AmyOes, 80)",80.0,AmyOes
55215,DANIEL BEHNKE,"(daniellie, 80)",80.0,daniellie
55216,WILLIAM LENNOX,"(lisacm, 80)",80.0,lisacm
55217,STAN MANEL,"(aelst, 80)",80.0,aelst


In [13]:
runnerdata_with_usernames.to_csv('~/Desktop/fuzzywuzzy.csv')